In [36]:
import subprocess
# cmd0 = "sudo add-apt-repository ppa:ubuntugis/ubuntugis-unstable"
# print(os.system(cmd0))
# cmd1 = "sudo apt-get update"
# print(os.system(cmd1))
# cmd2 = "sudo apt-get install libgdal-dev"
# print(os.system(cmd2))
# cmd3 = "virtualenv gdalenv"
# print(os.system(cmd3))
# cmd4 = "source gdal/bin/activate"
# print(os.system(cmd4))
# cmd5 = "pip install --no-install GDAL"
# print(os.system(cmd5))
# cmd6 = "python setup.py build_ext --include-dirs=/usr/include/gdal/"
# print(os.system(cmd6))
# cmd7 = "pip install gdal --global-option=build_ext --global-option='-I/usr/include/gdal/'"
# print(os.system(cmd7))

# cmd0 = "pip install GDAL==$(gdal-config --version) --global-option=build_ext --global-option='-I/usr/include/gdal' "
cmd0 = "conda install gdal"
print(os.system(cmd0))
print(subprocess.check_output(["gdalinfo", "--version"]))
# from osgeo import gdal
# import gdal

0
b'GDAL 3.0.4, released 2020/01/28\n'


#### Ensure that you are in the correct directory:

In [37]:
if os.path.isfile("README.md"):
    os.chdir("./{{cookiecutter._project_name}}")

In [38]:
import json
import numpy as np
import os
import urllib.request as urllib
import zipfile
import subprocess
import sys

from PIL import Image

import algorithm_rgb

#### Define the version number of your algorithm. Consider using [Semantic Versioning](https://semver.org/)

In [5]:
print(algorithm_rgb.VERSION)
print(algorithm_rgb.hello)
algorithm_rgb.hello = "goodbye"
print(algorithm_rgb.hello)
# os.system("nano algorithm_rgb.py")

1.0
goodbye
goodbye


#### Provide information on the creator and contributors of this algorithm

In [ ]:
algorithm_rgb.ALGORITHM_AUTHOR = 'Unknown'
algorithm_rgb.ALGORITHM_AUTHOR_EMAIL = 'author@example.com'
algorithm_rgb.ALGORITHM_CONTRIBUTORS = [""]

#### Name and describe your algorithm

In [ ]:
algorithm_rgb.ALGORITHM_NAME = 'default'
algorithm_rgb.ALGORITHM_DESCRIPTION = ''

#### Provide citation information for algorithm publication. This includes the citation author, the citation title, and the citation year

In [ ]:
algorithm_rgb.CITATION_AUTHOR = ''
algorithm_rgb.CITATION_TITLE = ''
algorithm_rgb.CITATION_YEAR = ''

#### Include the name(s) of the variable(s) used in the algorithm, separated by commas

In [ ]:
algorithm_rgb.VARIABLE_NAMES = 'size of image channels'

#### Include the units and labels of the variables, matching the order of VARIABLE_NAMES, also separated by commas. VARIABLE_LABELS is an optional field and can be left empty.

In [ ]:
algorithm_rgb.VARIABLE_UNITS = 'pixels'
algorithm_rgb.VARIABLE_LABELS = ''

#### Optional override for the generation of a BETYdb compatible csv file. Set to False to suppress the creation of a compatible file

In [ ]:
algorithm_rgb.WRITE_BETYDB_CSV = True

#### Optional override for the generation of a TERRA REF Geostreams compatible csv file. Set the variable to False to suppress the creation of a compatible file

In [ ]:
algorithm_rgb.WRITE_GEOSTREAMS_CSV = True

#### Change the url below to get images from a different source. This assumes that the image files are contained within a .zip file. The code below will add the sample_plot_images chosen to the system

In [ ]:
url = "https://de.cyverse.org/dl/d/4108BB75-AAA3-48E1-BBD4-E10B06CADF54/sample_plot_images.zip"

if os.path.isdir("sample_plot_images"):
    shutil.rmtree("sample_plot_images")
os.mkdir("sample_plot_images")
if not os.path.isfile("sample_plot_images.zip"):
    urllib.urlretrieve(url, "sample_plot_images.zip")
zipfile.ZipFile("sample_plot_images.zip", 'r').extractall("sample_plot_images")
os.remove("sample_plot_images.zip")

#### Define your calculate() function

In [ ]:
def _calculate(pxarray: np.ndarray):
    """Calculates one or more values from plot-level RGB data
    Arguments:
        pxarray: Array of RGB data for a single plot
    Return:
        Returns one or more calculated values
    """
    # ALGORITHM: replace the following lines with your algorithm
    channel_size = pxarray[:, :, 1].size*5

    # RETURN: replace the following return with your calculated values. Be sure to order them as defined in VARIABLE_NAMES above
    return channel_size

alg.calculate = _calculate

#### Test the calculate() function on the sample plot images located in the sample_plot_images folder

In [ ]:
for filename in os.listdir("sample_plot_images"):
    img = Image.open("sample_plot_images/" + filename)
    img_arr = np.array(img)
    print(alg.calculate(img_arr))

#### Next Generate your Dockerfile by running the generate.py script

In [ ]:
cmd0 = "python generate.py"
os.system(cmd0)

In [ ]:
#### If there are leftover files from previous runs, delete them

In [ ]:
filelist = ["result.json", "rgb_plot.csv", "rgb_plot_betydb.csv", "rgb_plot_geo.csv"]
for file in filelist:
    if os.path.isfile(file):
        os.remove(file)

#### Now build the dockerfile (Currently this will have a default project name and project version)

In [ ]:
cmd = "docker build -t " + alg.ALGORITHM_NAME + ":" + alg.VERSION + " ."
os.system(cmd)

#### Next run the dockerfile for testing

In [ ]:
cmd = 'docker run --rm --mount "src=`pwd`,target=/mnt,type=bind" default:latest --working_space "/mnt"'
for filename in os.listdir("sample_plot_images"):
    cmd += ' "/mnt/sample_plot_images/' + filename + '"'
    
os.system(cmd)

#### Make sure that the correct files are generated and contain appropriate results

In [ ]:
filelist = ["result.json", "rgb_plot.csv", "rgb_plot_betydb.csv", "rgb_plot_geo.csv"]
for file in filelist:
    assert os.path.isfile(file)
    if (file == "result.json"):
        result = json.load(open(file))["unknown"]
        assert result['version'] == algorithm_rgb.VERSION
        assert result['traits'] == algorithm_rgb.VARIABLE_NAMES
        assert result['units'] == algorithm_rgb.VARIABLE_UNITS
        assert result['labels'] == algorithm_rgb.VARIABLE_LABELS
        assert result['files_processed'] == str(len(os.listdir("sample_plot_images")))
        assert result['lines_written'] == str(len(os.listdir("sample_plot_images")))
        if (alg.WRITE_GEOSTREAMS_CSV == True):
            assert result['wrote_geostreams'] == "Yes"
        else:
            assert result['wrote_geostreams'] == "No"
        if (alg.WRITE_BETYDB_CSV == True):
            assert result['wrote_betydb'] == "Yes"
        else: 
            assert result['wrote_betydb'] == "No"

### The following test whether the fields and algorithm are in the correct format